# DsDA Controller

Creates multiple dataset and runs them through the DsDA method

In [ ]:
%matplotlib inline

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import sys
import scipy.stats
import pylab as plt
from IPython import display
import pylab as plt
import glob
import os

In [ ]:
base_dir = os.path.join('..', '..')
sys.path.append(base_dir)
dsda_dir = os.getcwd()

In [ ]:
from vimms.Chemicals import *
from vimms.Chromatograms import *
from vimms.MassSpec import *
from vimms.Controller import *
from vimms.Common import *
from vimms.DataGenerator import *
from vimms.DsDA import *

In [ ]:
 set_log_level_warning()
# set_log_level_info()
# set_log_level_debug()

In [ ]:
ps = load_obj(os.path.join(base_dir, 'examples', 'example_data', 'peak_sampler_mz_rt_int_19_beers_fragmentation.p'))

#  Initial Settings

In [ ]:
isolation_window = 1   # the isolation window in Dalton around a selected precursor ion
min_ms1_intensity = 1.75E5 # minimum ms1 intensity to fragment
min_rt = 3*60
max_rt = 21*60
N = 4
rt_tol = 15
mz_tol = 10

In [ ]:
beer_dir = os.path.join(base_dir, 'examples', 'example_data', 'DsDA', 'DsDA_Beer', 'beer_t10_simulator_files')

# Run DsDA on Beer Data

In [ ]:
schedule_dir = os.path.join(dsda_dir, 'DsDA_Beer', 'settings')
data_dir = os.path.join(dsda_dir, 'DsDA_Beer', 'Data')
controllers_dir = os.path.join(dsda_dir, 'DsDA_Beer', 'controllers')

Before running the next code, open and run the R script in '\\DsDa_Beer' in a new R window.

Requirements:
- Ensure that XCMS3 has been installed: https://bioconductor.org/packages/release/bioc/html/xcms.html.

In [ ]:
initial_schedule = get_schedule(1, os.path.join(dsda_dir, 'DsDA_Beer'))
print("start")
for n in range(1,20,1):
    dataset = load_obj(os.path.join(beer_dir, 'beer_%d.p' % n))
    if n == 1:
        mass_spec = IndependentMassSpectrometer(POSITIVE, dataset, density=ps.density_estimator, schedule_file=initial_schedule)
        controller = TopNController(mass_spec, N, isolation_window, mz_tol, rt_tol, min_ms1_intensity)
        controller.run()
    else:
        last_schedule = get_schedule(n-1, schedule_dir)
        mass_spec = DsDAMassSpec(POSITIVE, dataset, density=ps.density_estimator)
        controller = DsDAController(mass_spec, 1, 0.5, 15, 2E5)
        controller.run(last_schedule)
    print("Processed ", n+1, " files")
    controller.write_mzML('my_analysis', os.path.join(data_dir, 'beer_dsda_%d.mzML' % n))
    save_obj(controller, os.path.join(controllers_dir, 'beer_dsda_%d.p' % n))
    print("Waiting for R to process .mzML files")

# Run DsDA MaxDepth on Beer Data

In [ ]:
schedule_dir = os.path.join(dsda_dir, 'DsDA_Beer_MaxDepth', 'settings')
data_dir = os.path.join(dsda_dir, 'DsDA_Beer_MaxDepth', 'Data')
controllers_dir = os.path.join(dsda_dir, 'DsDA_Beer_MaxDepth', 'controllers')

Before running the next code, open and run the R script in '\\DsDa_Beer' in a new R window

In [ ]:
initial_schedule = get_schedule(1, os.path.join(dsda_dir, 'DsDA_Beer_MaxDepth'))
print("start")
for n in range(1,20,1):
    dataset = load_obj(os.path.join(beer_dir, 'beer_%d.p' % n))
    if n == 1:
        mass_spec = IndependentMassSpectrometer(POSITIVE, dataset, density=ps.density_estimator, schedule_file=initial_schedule)
        controller = TopNController(mass_spec, N, isolation_window, mz_tol, rt_tol, min_ms1_intensity)
        controller.run()
    else:
        last_schedule = get_schedule(n-1, schedule_dir)
        mass_spec = DsDAMassSpec(POSITIVE, dataset, density=ps.density_estimator)
        controller = DsDAController(mass_spec, 1, 0.5, 15, 2E5)
        controller.run(last_schedule)
    print("Processed ", n+1, " files")
    controller.write_mzML('my_analysis', os.path.join(data_dir, 'beer_dsda_md_%d.mzML' % n))
    save_obj(controller, os.path.join(controllers_dir, 'beer_dsda_md_%d.p' % n))
    print("Waiting for R to process .mzML files")

# T4 on Beer Data

In [ ]:
t4_controllers_dir = os.path.join(dsda_dir + 'T4_Beer')
os.chdir(beer_dir)
file_names = glob.glob("*.p")

In [ ]:
schedule = get_schedule(1, t4_controllers_dir)
set_log_level_warning()
for n in range(len(file_names)):
    dataset = load_obj(file_names[n])
    mass_spec = IndependentMassSpectrometer(POSITIVE, dataset, density=ps.density_estimator, schedule_file=schedule)
    controller = TopNController(mass_spec, N, isolation_window, mz_tol, rt_tol, min_ms1_intensity)
    controller.run()
    print("Processed ", n+1, " files")
    save_obj(controller, os.path.join(t4_controllers_dir, 't4_beer_%d.p' % n))

# Code to Analyse Beer Data

In [ ]:
dsda_controller_directory = os.path.join(dsda_dir, 'DsDA_Beer', 'controllers')
dsda_maxdepth_controller_directory = os.path.join(dsda_dir + '\\DsDA_Beer_MaxDepth\\controllers'
aligned_beer_chemicals = dsda_dir + '\\Aligned_Peaks\\beer_aligned_features_ms1.csv'
params_t4 = create_frag_dicts(t4_controller_directory, aligned_beer_chemicals, 1.75E5)
params_dsda = create_frag_dicts(dsda_controller_directory, aligned_beer_chemicals, 1.75E5)
params_dsda_maxdepth = create_frag_dicts(dsda_maxdepth_controller_directory, aligned_beer_chemicals, 1.75E5)

In [ ]:
beer_chemicals_found_results_t4 = multi_sample_fragmentation_performance_aligned(params_t4)
beer_chemicals_found_results_dsda = multi_sample_fragmentation_performance_aligned(params_dsda)
beer_chemicals_found_results_dsda_maxdepth = multi_sample_fragmentation_performance_aligned(params_dsda_maxdepth)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(range(1,20,1),np.array(beer_chemicals_found_results_dsda_maxdepth),label="DsDA MaxDepth")
plt.plot(range(1,20,1),np.array(beer_chemicals_found_results_dsda),label="DsDA")
plt.plot(range(1,20,1),np.array(beer_chemicals_found_results_t4),label="T4")
plt.legend(loc='lower right')
ax = plt.gca()
for l in ax.lines:
    plt.setp(l, linewidth=5)
plt.tight_layout()